# Use Model Context Protocol (MCP) as tools with Strands Agent

## Overview
The [Model Context Protocol (MCP)](https://modelcontextprotocol.io/introduction) is an open protocol that standardizes how applications provide context to Large Language Models (LLMs). Strands AI SDK integrates with MCP to extend agent capabilities through external tools and services.

MCP enables communication between agents and MCP servers that provide additional tools. The Strands Agent SDK includes built-in support for connecting to MCP servers and using their tools.

In this example we will show you how to use MCP tools on your Strands Agent. We will use the [AWS Documentation MCP server](https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server/) which provides tools to access AWS documentation, search for content, and get recommendations. This MCP server has 3 main features:

- **Read Documentation**: Fetch and convert AWS documentation pages to markdown format
- **Search Documentation**: Search AWS documentation using the official search API
- **Recommendations**: Get content recommendations for AWS documentation pages



## Agent Details
<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|Feature used        |MCP Tools                                          |
|Agent Structure     |Single agent architecture                          |

</div>

## Architecture

<div style="text-align:center">
    <img src="images/architecture.png" width="65%" />
</div>

## Key Features
* **Single agent architecture**: this example creates a single agent that interacts with MCP tools
* **MCP tools**: Integration of MCP tools with your agent

## Setup and prerequisites

### Prerequisites
* Python 3.10+
* AWS account
* Anthropic Claude 3.7 enabled on Amazon Bedrock

Let's now install the requirement packages for our Strands Agent agent

In [1]:
# installing pre-requisites
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 182.0 MB/s eta 0:00:00


### Importing dependency packages

Now let's import the dependency packages

In [2]:
import threading
import time
from datetime import timedelta

from mcp import StdioServerParameters, stdio_client
from mcp.client.streamable_http import streamablehttp_client
from mcp.server import FastMCP
from strands import Agent
from strands.tools.mcp import MCPClient

### Connect to MCP server using stdio transport

[Transports](https://modelcontextprotocol.io/specification/2025-03-26/basic/transports) in MCP provide the foundations for communication between clients and servers. It handles the underlying mechanics of how messages are sent and received. At the moment there are three standards transport implementations built-in in MCP:

- **Standard Input/Output (stdio)**: enables communication through standard input and output streams. It is particularly useful for local integrations and command-line tools
- **Streamable HTTP**: this replaces the HTTP+SSE transport from previous protocol version. In the Streamable HTTP transport, the server operates as an independent process that can handle multiple client connections. This transport uses HTTP POST and GET requests. Server can optionally make use of Server-Sent Events (SSE) to stream multiple server messages. This permits basic MCP servers, as well as more feature-rich servers supporting streaming and server-to-client notifications and requests.
- **SSE**: legacy transport for HTTP-based MCP servers that use Server-Sent Events transport  

Overall, you should use stdio for building command-line tools, implementing local integrations and working with shell scripts. You should use Streamable HTTP transports when you need a flexible and efficient way for AI agents to communicate with tools and services, especially when dealing with stateless communication or when minimizing resource usage is crucial.

You can also use **custom transports** implementation for your specific needs. 


Let's now connect to the MCP server using stdio transport. First of all, we will use the class `MCPClient` to connect to the [AWS Documentation MCP Server](https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server/). This server provides tools to access AWS documentation, search for content, and get recommendations.

In [3]:
# Connect to an MCP server using stdio transport
stdio_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

#### Setup agent configuration and invoke it

Next we will set our agent configuration using the tools from the `stdio_mcp_client` object we just created. To do so, we need to list the tools available in the MCP server. We can use the `list_tools_sync` method for it. 

After that, we will ask a question to our agent.

In [4]:
# Create an agent with MCP tools
with stdio_mcp_client:
    # Get the tools from the MCP server
    tools = stdio_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        tools=tools)

    response = agent("What is Amazon Bedrock pricing model. Be concise.")

I'll look up the Amazon Bedrock pricing model for you. Let me search the AWS documentation.
Tool #1: search_documentation
Now, let me retrieve the specific pricing information for Amazon Bedrock:
Tool #2: read_documentation
Based on the AWS documentation, here's the concise overview of Amazon Bedrock's pricing model:

## Amazon Bedrock Pricing Model

- **Pay-as-you-go basis**: You are charged only for the services you use.
- **Based on token usage**: Pricing depends on the volume of input tokens and output tokens processed.
- **Model-specific pricing**: Different foundation models have different pricing rates.
- **Two throughput options**:
  1. On-demand usage (standard pay-as-you-go)
  2. Provisioned Throughput (purchased in advance for increased capacity and potentially lower costs at scale)

Pricing details for specific models can be found on the Amazon Bedrock pricing page (aws.amazon.com/bedrock/pricing) or in the Amazon Bedrock console under "Model providers."

### Connect to MCP server using Streamable HTTP

Let's now connect to the MCP server using Streamable HTTP transport. First let's start a simple MCP server using Streamable HTTP transport. 

For this example we will create our own MCP server. The architecture will look as following

<div style="text-align:center">
    <img src="images/architecture_2.png" width="65%" />
</div>

In [5]:
# Create an MCP server
mcp = FastMCP("Calculator Server")

# Define a tool


@mcp.tool(description="Calculator tool which performs calculations")
def calculator(x: int, y: int) -> int:
    return x + y


@mcp.tool(description="This is a long running tool")
def long_running_tool(name: str) -> str:
    time.sleep(25)
    return f"Hello {name}"


def main():
    mcp.run(transport="streamable-http", mount_path="mcp")

Let's now start a thread with the `streamable-http` server

In [6]:
thread = threading.Thread(target=main)
thread.start()

INFO:     Started server process [5395]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:41206 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:41214 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:41228 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:41236 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:41246 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:41250 - "DELETE /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:41260 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:41270 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:41286 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:41302 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:41316 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:41318 - "DELETE /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:41322 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:41340 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:41330 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:41352 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:49166 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:49168 - "

#### Integrating Streamable HTTP client with Agent

Now let's use `streamablehttp_client` integrate this server with a simple agent. 

In [7]:
def create_streamable_http_transport():
    return streamablehttp_client("http://localhost:8000/mcp")


streamable_http_mcp_client = MCPClient(create_streamable_http_transport)

#### Setup agent configuration and invoke it

Next we will set our agent configuration using the tools from the `streamable_http_mcp_client` object we just created. To do so, we need to list the tools available in the MCP server. We can use the `list_tools_sync` method for it. 

After that, we will ask a question to our agent.

In [8]:
with streamable_http_mcp_client:
    tools = streamable_http_mcp_client.list_tools_sync()

    agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        tools=tools)

    response = str(agent("What is 2 + 2?"))

[07/30/25 20:03:11] INFO     StreamableHTTP session manager started                  ]8;id=856968;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=198410;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py#110\110]8;;\

                    INFO     Created new transport with session ID:                  ]8;id=975001;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=384724;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py#233\233]8;;\
                             2e4954886cde4e0886f47245e2c1608a                                                      

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=43315;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=595074;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Received session ID: 2e4954886cde4e0886f47245e2c1608a           ]8;id=867442;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=780551;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py#134\134]8;;\

                    INFO     Negotiated protocol version: 2025-06-18                         ]8;id=788200;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=78578;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py#146\146]8;;\

...

[07/30/25 20:03:14] INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=715365;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=870082;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Processing request of type CallToolRequest                               ]8;id=291321;file:///opt/conda/lib/python3.12/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=643542;file:///opt/conda/lib/python3.12/site-packages/mcp/server/lowlevel/server.py#625\625]8;;\

The answer to 2 + 2 is 4.

[07/30/25 20:03:16] INFO     Terminating session: 2e4954886cde4e0886f47245e2c1608a           ]8;id=177876;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http.py\streamable_http.py]8;;\:]8;id=268068;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http.py#630\630]8;;\

                    INFO     HTTP Request: DELETE http://localhost:8000/mcp "HTTP/1.1 200 OK"       ]8;id=520217;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=653002;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

### Direct Tool Invocation

While tools are typically invoked by the agent based on user requests, you can also call MCP tools directly. This can be useful for workflow scenarios where you orchestrate multiple tools together.

In [9]:
query = {"x": 10, "y": 20}

with streamable_http_mcp_client:
    # direct tool invocation
    result = streamable_http_mcp_client.call_tool_sync(
        tool_use_id="tool-123", name="calculator", arguments=query
    )

    # Process the result
    print(f"Calculation result: {result['content'][0]['text']}")

                    INFO     Created new transport with session ID:                  ]8;id=984687;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=678854;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py#233\233]8;;\
                             866ab3aea6304316baf5ba19da0860a9                                                      

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=739721;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=443782;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Received session ID: 866ab3aea6304316baf5ba19da0860a9           ]8;id=977084;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=592663;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py#134\134]8;;\

                    INFO     Negotiated protocol version: 2025-06-18                         ]8;id=137888;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=137944;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py#146\146]8;;\

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 202 Accepted"   ]8;id=74027;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=886631;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

...

                    INFO     Processing request of type ListToolsRequest                              ]8;id=312995;file:///opt/conda/lib/python3.12/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=745741;file:///opt/conda/lib/python3.12/site-packages/mcp/server/lowlevel/server.py#625\625]8;;\

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=380982;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=593270;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

Calculation result: 30


                    INFO     Terminating session: 866ab3aea6304316baf5ba19da0860a9           ]8;id=434748;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http.py\streamable_http.py]8;;\:]8;id=993465;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http.py#630\630]8;;\

                    INFO     HTTP Request: DELETE http://localhost:8000/mcp "HTTP/1.1 200 OK"       ]8;id=572526;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=389493;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

You can optionally also provide `read_timeout_seconds` while calling an MCP server tool to avoid it running for too long

In [10]:
with streamable_http_mcp_client:
    try:
        result = streamable_http_mcp_client.call_tool_sync(
            tool_use_id="tool-123",
            name="long_running_tool",
            arguments={"name": "Amazon"},
            read_timeout_seconds=timedelta(seconds=30),
        )

        if result["status"] == "error":
            print(f"Tool execution failed: {result['content'][0]['text']}")
        else:
            print(f"Tool execution succeeded: {result['content'][0]['text']}")
    except Exception as e:
        print(f"Tool call timed out or failed: {str(e)}")

                    INFO     Created new transport with session ID:                  ]8;id=187523;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=120646;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py#233\233]8;;\
                             3b565cf0d1644fc49545bb210d96ebcc                                                      

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=541972;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=954375;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Received session ID: 3b565cf0d1644fc49545bb210d96ebcc           ]8;id=228558;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=263231;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py#134\134]8;;\

                    INFO     Negotiated protocol version: 2025-06-18                         ]8;id=814285;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=889483;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py#146\146]8;;\

                    INFO     HTTP Request: GET http://localhost:8000/mcp "HTTP/1.1 200 OK"          ]8;id=738795;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=533573;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

...

[07/30/25 20:03:41] INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=681244;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=157703;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Processing request of type ListToolsRequest                              ]8;id=909779;file:///opt/conda/lib/python3.12/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=909888;file:///opt/conda/lib/python3.12/site-packages/mcp/server/lowlevel/server.py#625\625]8;;\

Tool execution succeeded: Hello Amazon


                    INFO     Terminating session: 3b565cf0d1644fc49545bb210d96ebcc           ]8;id=90461;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http.py\streamable_http.py]8;;\:]8;id=446352;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http.py#630\630]8;;\

                    INFO     HTTP Request: DELETE http://localhost:8000/mcp "HTTP/1.1 200 OK"       ]8;id=238113;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=925444;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

### Interacting with multiple MCP servers

With Strands Agents you can also interact with multiple MCP servers using the same agent and configure tools setups such as the max number of tools that can be used in parallel (`max_parallel_tools`). Let's create a new agent to showcase this configuration:

<div style="text-align:center">
    <img src="images/architecture_3.png" width="85%" />
</div>

In this agent, we will again use the AWS Documentation MCP server and we will also use the [AWS CDK MCP Server](https://awslabs.github.io/mcp/servers/cdk-mcp-server/) which helps with AWS Cloud Development Kit (CDK) best practices, infrastructure as code patterns and security compliance with CDK Nag.

First let's connect to the two MCP servers using the stdio transport

In [11]:
# Connect to an MCP server using stdio transport
aws_docs_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

# Connect to an MCP server using stdio transport
cdk_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(command="uvx", args=["awslabs.cdk-mcp-server@latest"])
    )
)

#### Create Agent with MCP servers

Next we will create the agent with the tools from both MCP servers

In [12]:
# Create an agent with MCP tools
with aws_docs_mcp_client, cdk_mcp_client:
    # Get the tools from the MCP server
    tools = aws_docs_mcp_client.list_tools_sync() + cdk_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        tools=tools)

    response = agent(
        "What is Amazon Bedrock pricing model. Be concise. Also what are the best practices related to CDK?"
    )

I'll answer both of your questions about Amazon Bedrock pricing and CDK best practices. Let me get the information for you.

For Amazon Bedrock pricing:
Tool #1: search_documentation
Now, let me get more detailed information about Amazon Bedrock pricing:
Tool #2: read_documentation
Let me also get information about CDK best practices:
Tool #3: CDKGeneralGuidance
## Amazon Bedrock Pricing Model

Amazon Bedrock uses a pay-as-you-go pricing model based on:

1. **Token-based pricing**: You pay for the volume of input tokens and output tokens processed.
2. **Model-specific rates**: Different foundation models have different pricing rates.
3. **Provisioned Throughput option**: You can purchase dedicated capacity for consistent performance and potentially lower costs at scale.

Each model provider and model version has its own specific pricing, which you can find on the Model providers page in the Amazon Bedrock console. You're only charged for the AWS services you actually use.

## CDK Best 

### Congratulations!

In this notebook you learned how to connect with MCP servers using Strands Agent and two MCP transport protocols: stdio and Streamable HTTP. You also learned how to connect multiple MCP servers to the same agent. Next, let's see how to use different models with your agent